In [38]:
import pandas as pd
import numpy as np
file_path = r'C:\Users\harsh\OneDrive\Documents\AI job market\data\processed\cleaned_postings_with_skills.csv'
df = pd.read_csv(file_path)

In [39]:
df['normalized_salary'] = pd.to_numeric(df['normalized_salary'], errors='coerce')
df.loc[df['normalized_salary'] == 0, 'normalized_salary'] = np.nan
df.dropna(subset=['normalized_salary'], inplace=True)
df.head(50)


,job_id,company_name,title,description,skills_desc,normalized_salary,formatted_experience_level,formatted_work_type,remote_allowed,posting_domain,location,listed_time,zip_code,extracted_skills
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,Requirements: \n\nWe are seeking a College or ...,38480.0,NaN,Full-time,False,NaN,"Princeton, NJ",1.713400e+12,8540.0,NaN
1,1829192,Unknown,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",NaN,83200.0,NaN,Full-time,False,NaN,"Fort Collins, CO",1.712860e+12,8540.0,NaN
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,We are currently accepting resumes for FOH - A...,55000.0,NaN,Full-time,False,NaN,"Cincinnati, OH",1.713280e+12,8540.0,NaN
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,This position requires a baseline understandin...,157500.0,NaN,Full-time,False,NaN,"New Hyde Park, NY",1.712900e+12,8540.0,NaN
4,35982263,Unknown,Service Technician,Looking for HVAC service tech with experience ...,NaN,70000.0,NaN,Full-time,False,NaN,"Burlington, IA",1.713450e+12,8540.0,NaN
5,91700727,Downtown Raleigh Alliance,Economic Development and Planning Intern,Job summary:The Economic Development & Plannin...,NaN,35360.0,NaN,Internship,False,NaN,"Raleigh, NC",1.713460e+12,8540.0,NaN
6,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,NaN,180000.0,NaN,Contract,True,NaN,United States,1.712860e+12,8540.0,NaN
7,112576855,Unknown,Building Engineer,Summary: Due to the pending retirement of our ...,NaN,105000.0,NaN,Full-time,False,NaN,"San Francisco, CA",1.712440e+12,8540.0,NaN
9,2264355,Bay West Church,Worship Leader,It is an exciting time to be a part of our chu...,"Knowledge, Skills and Abilities: 1. Proficient...",4200.0,NaN,Part-time,False,NaN,"Palm Bay, FL",1.712460e+12,8540.0,NaN
16,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,52000.0,NaN,Full-time,False,NaN,"Cleveland, OH",1.712860e+12,8540.0,NaN


In [40]:
df = df.drop(
    columns=[
        'company_name',
        'description',
        'skills_desc',
        'posting_domain',
        'location',
        'listed_time',
        'zip_code'
    ],
    errors='ignore'
)

In [41]:
df['extracted_skills'] = (
    df['extracted_skills']
    .astype('string')
    .str.replace('|', ' ', regex=False)
    .fillna('no_skills_listed')
)
df.head(42)
df.to_csv(r'C:\Users\harsh\OneDrive\Documents\AI job market\ML_prediction/final_cleaned_dataset.csv', index=False)

In [42]:
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import xgboost as xgb

In [43]:
column = ['formatted_experience_level', 'title','formatted_work_type', 'remote_allowed','extracted_skills']
X_train , X_test , y_train , y_test = train_test_split(df[column], df['normalized_salary'], test_size=0.2, random_state=42)

In [44]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
column1 = ['formatted_experience_level', 'formatted_work_type', 'remote_allowed']
meta_train_encoded = encoder.fit_transform(X_train[column1])
meta_test_encoded = encoder.transform(X_test[column1])
feature_names_out = encoder.get_feature_names_out(column1)

#is a sparse matrix so cannot be converted directly to dataframe
# df_enc = pd.DataFrame(encoded_data,columns=feature_names_out)
# df_enc.head(50)
# X , y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)

In [45]:
vectorizer = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()
tfidf_train_matrix = vectorizer.fit_transform(X_train['extracted_skills'])
tfidf_train_matrix2 = vectorizer2.fit_transform(X_train['title'])
tfidf_test_matrix = vectorizer.transform(X_test['extracted_skills'])
tfidf_test_matrix2 = vectorizer2.transform(X_test['title'])
print(vectorizer.get_feature_names_out())

['ab' 'acr' 'actions' 'adls' 'agile' 'ai' 'airflow' 'aks' 'amazon'
 'angular' 'ansible' 'apache' 'api' 'artificial' 'asp' 'athena'
 'augmented' 'avro' 'aws' 'azure' 'bash' 'bayes' 'bi' 'bigquery'
 'bigtable' 'bitbucket' 'boot' 'bootstrap' 'cassandra' 'cd' 'chroma' 'ci'
 'circleci' 'cloud' 'cloudformation' 'cloudwatch' 'clustering' 'computer'
 'confluence' 'connect' 'core' 'cosmos' 'css' 'cypress' 'dagster' 'data'
 'database' 'datadog' 'dataflow' 'db' 'dbt' 'deep' 'delta' 'deployment'
 'design' 'devops' 'dimensional' 'django' 'docker' 'dynamodb' 'ecr' 'ecs'
 'eks' 'elasticsearch' 'elk' 'elt' 'embeddings' 'encryption' 'engineering'
 'etl' 'excel' 'experiment' 'express' 'face' 'factory' 'fargate' 'fastapi'
 'feature' 'fivetran' 'flask' 'flink' 'forecasting' 'functions' 'gateway'
 'gcp' 'gcs' 'genai' 'generation' 'generative' 'git' 'github' 'gitlab'
 'gke' 'glue' 'go' 'golang' 'google' 'grafana' 'graphql' 'grpc' 'hadoop'
 'hdfs' 'helm' 'hive' 'html' 'hugging' 'iam' 'iceberg' 'informatica'


In [46]:
model = XGBRegressor()

In [47]:
print(tfidf_train_matrix.shape)

print(meta_train_encoded.shape)

print(tfidf_test_matrix.shape)

print(meta_test_encoded.shape)

(28847, 274)
(28847, 16)
(7212, 274)
(7212, 16)


In [48]:
from scipy.sparse import hstack

# Combine (skills tfidf + meta onehot) into one sparse matrix
X_train_final = hstack([tfidf_train_matrix,tfidf_train_matrix2, meta_train_encoded]).tocsr()
X_test_final  = hstack([tfidf_test_matrix, tfidf_test_matrix2, meta_test_encoded]).tocsr()

print("X_train_final:", X_train_final.shape)
print("X_test_final :", X_test_final.shape)


X_train_final: (28847, 8560)
X_test_final : (7212, 8560)


In [49]:
# Split train into train/val
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_final, y_train, test_size=0.15, random_state=42
)

# DMatrix (works with scipy sparse)
dtrain = xgb.DMatrix(X_tr, label=y_tr)
dval   = xgb.DMatrix(X_val, label=y_val)
dtest  = xgb.DMatrix(X_test_final, label=y_test)

# Start from your tuned params, but remove sklearn-only keys if present
params = {
    "objective": "reg:squarederror",
    "tree_method": "hist",
    "max_depth": 7,
    "learning_rate": 0.2,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "gamma": 1,
    "reg_alpha": 0.0,
    "reg_lambda": 1.0,
    "seed": 42
}

# Use a large num_boost_round + early stopping to choose best iteration
evals = [(dtrain, "train"), (dval, "val")]

bst = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=3000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=False
)

# Predict using best_iteration automatically
y_pred = bst.predict(dtest)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("Best iteration:", bst.best_iteration)
print(f"RMSE (USD): {rmse:,.2f}")
print(f"MAE  (USD): {mae:,.2f}")
print(f"R^2        : {r2:.4f}")


Best iteration: 0
RMSE (USD): 4,887,818.93
MAE  (USD): 224,465.58
R^2        : -0.9840


In [51]:
print("Best iteration:", bst.best_iteration)

# 4) Evaluate on test
y_pred = bst.predict(dtest)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"RMSE (USD): {rmse:,.2f}")
print(f"MAE  (USD): {mae:,.2f}")
print(f"R^2       : {r2:.4f}")

# 5) Save model + preprocessors
joblib.dump(encoder, r"C:\Users\harsh\OneDrive\Documents\AI job market\models\encoders\ohe_encoder.joblib")
joblib.dump(vectorizer, r"C:\Users\harsh\OneDrive\Documents\AI job market\models\encoders\tfidf_skills.joblib")
joblib.dump(vectorizer2, r"C:\Users\harsh\OneDrive\Documents\AI job market\models\encoders\tfidf_title.joblib")

bst.save_model(r"C:\Users\harsh\OneDrive\Documents\AI job market\models\salary_xgb_model.json")

print("Saved: ohe_encoder.joblib, tfidf_skills.joblib, tfidf_title.joblib, salary_xgb_model.json")

Best iteration: 0
RMSE (USD): 4,887,818.93
MAE  (USD): 224,465.58
R^2       : -0.9840
Saved: ohe_encoder.joblib, tfidf_skills.joblib, tfidf_title.joblib, salary_xgb_model.json
